In [6]:
import open3d as o3d
import copy
import math
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import time

In [7]:
import math
import numpy as np 

def readData(filename):
    data = np.loadtxt(filename)
    #data[i].reshape(3,4)
    return data 

def readPointCloud(filename):
    pcl = np.fromfile(filename, dtype=np.float32,count=-1)
    pcl = pcl.reshape([-1,4])
    return pcl 

def EulerAngles_To_RotationMatrix(theta) :    
    R_x = np.array([[1,0,0],
                    [0,math.cos(theta[2]),-math.sin(theta[2]) ],
                    [0,math.sin(theta[2]),math.cos(theta[2])  ]
                    ])
    R_y = np.array([[math.cos(theta[1]),0,math.sin(theta[1])  ],
                    [0,1,0],
                    [-math.sin(theta[1]),0,math.cos(theta[1]) ]
                    ])
    R_z = np.array([[math.cos(theta[0]),-math.sin(theta[0]),0 ],
                    [math.sin(theta[0]),math.cos(theta[0]),0],
                    [0,0,1]
                    ])
    R = np.dot(R_z, np.dot( R_y, R_x ))
    return R

In [8]:
me=EulerAngles_To_RotationMatrix([math.radians(90),math.radians(-90),0])
    
me2=np.zeros((3,4))
for p in range(3):
    for q in range(3):
         me2[p][q]=me[p][q]
print(me2.astype(int))

[[ 0 -1  0  0]
 [ 0  0 -1  0]
 [ 1  0  0  0]]


In [9]:

my_mat=readData('../dataset/01.txt')
final = o3d.geometry.PointCloud()
for i in range (77):
    st=[0,0,0,0,0,0]
    
    for j in range (6):
        st[5-j]=math.floor((i/(10**j))%10)
        
    st_me=''.join([str(elem) for elem in st]) 
    myfile="../dataset/01/"+st_me+".bin"
    yy=readPointCloud(myfile)

    yt=np.transpose(yy)
    me=EulerAngles_To_RotationMatrix([math.radians(90),math.radians(-90),0])
    
    me2=np.zeros((3,4))
    for p in range(3):
        for q in range(3):
             me2[p][q]=me[p][q]  # me2 - Lidar to camera
    yy=np.matmul(me2,yt)
    
    me2=yy.shape[1]
    yy.resize((4,me2))
    temp=my_mat[i]
    temp2=np.zeros((3,4))
    yy[3]=np.ones((1,me2))
    for r in range(3):
        for s in range(4):
            temp2[r][s]=temp[(r*4)+s]  # temp2 - camera to world
            
    yp=np.matmul(temp2,yy)
    yt1=np.transpose(yp)
    pcd = o3d.geometry.PointCloud()
    
    pcd.points = o3d.utility.Vector3dVector(yt1) # pcd registration
    final = final + pcd
    if(i==0):
        o3d.visualization.draw_geometries(
       [ pcd])


In [ ]:
# Final pcd downsampled

downpcd = final.voxel_down_sample(voxel_size=0.0001)
o3d.visualization.draw_geometries(
       [ downpcd])